# What is better, sym or antisym


In [4]:
import gc
import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats

#from sa_decomp_layer import SADecompLayer

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #disables GPU 
import matplotlib
import matplotlib.pyplot as plt

#tf.__version__
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp
from glob import glob


from tensorflow.keras.utils import to_categorical
from tensorflow.data import Dataset, AUTOTUNE
from imagenet_utils.imagenet_clsloc2 import clsloc
from imagenet_utils.load_images import load_images
from imagenet_utils.preprocess import preprocess

import keras
from keras import layers

from keras import backend as K

In [5]:
def reduce_min_or_max(tensor, axis=None):
    max = tf.reduce_max(tensor, axis=axis)
    min = tf.reduce_min(tensor, axis=axis)
    k = tf.where(tf.math.abs(min) > max, x=-1., y=1.)
    return tf.math.multiply(tf.reduce_max(tf.math.abs(tensor), axis=axis),  k) 

def mean_direction(angles):
    angles = (angles)
    n = angles.shape[0]
    print(n)

    C = np.sum(np.cos(angles))
    S = np.sum(np.sin(angles))
    print(C,S)
    if S > 0 and C > 0:
        return (np.arctan2(S,C))
    if C < 0:
        return (np.arctan2(S,C) + np.pi )
    if S<0 and C > 0:
        print(1)
        return (np.arctan2(S,C) + 2*np.pi)

def get_conv_layer(model, layer):
    conv_layers = []
    for l in model.layers:
        if 'conv2d' in str(type(l)).lower():
            conv_layers.append(l)
    layer = conv_layers[layer]
    return layer

def get_filter_and_bias(model, layer):

    conv_layers = []
    for l in model.layers:
        if 'conv2d' in str(type(l)).lower():
            conv_layers.append(l)
    layer = conv_layers[layer]

    # check for convolutional layer
    if 'conv' not in layer.name:
        raise ValueError('Layer must be a conv. layer')
    # get filter weights
    filters, biases = layer.get_weights()
    #print("biases shape : ", biases.shape)
    #print("filters shape : ", filters.shape)

    return (filters, biases)
    #print(layer.name, filters.shape)

# I use sobel to determine the dominant orientation of the filter
def getSobelAngle(f):

    s_h = sobel_h(f)
    s_v = sobel_v(f)

    return (np.arctan2(s_h,s_v))

def getSobelTF(f):

    sobel_h = np.array([[1., 2., 1.], [0., 0., 0.], [-1., -2., -1.]], dtype=np.float32).reshape((3, 3, 1, 1) )/-4
    sobel_v = np.array([[1., 0., -1.], [2., 0., -2.], [1., 0., -1.]], dtype=np.float32).reshape((3, 3, 1, 1))/-4    

    s_h = reduce_sum(multiply(f, sobel_h), axis=[0,1])
    s_v = reduce_sum(multiply(f, sobel_v), axis=[0,1])

    return (np.arctan2(s_h,s_v))



def getSymAntiSym(filter):

    #patches = extract_image_patches(filters, [1, k, k, 1],  [1, k, k, 1], rates = [1,1,1,1] , padding = 'VALID')
    #print(patches)
    mat_flip_x = np.fliplr(filter)

    mat_flip_y = np.flipud(filter)

    mat_flip_xy =  np.fliplr( np.flipud(filter))

    sum = filter + mat_flip_x + mat_flip_y + mat_flip_xy
    mat_sum_rot_90 = np.rot90(sum)
    
    return  (sum + mat_sum_rot_90) / 8, filter - ((sum + mat_sum_rot_90) / 8)

def getSymAntiSymTF(filter):

    #patches = extract_image_patches(filters, [1, k, k, 1],  [1, k, k, 1], rates = [1,1,1,1] , padding = 'VALID')
    #print(patches)
    a = filter[0,0,:,:]
    b = filter[0,1,:,:]
    c = filter[0,2,:,:]
    d = filter[1,0,:,:]
    e = filter[1,1,:,:]
    f = filter[1,2,:,:]
    g = filter[2,0,:,:]
    h = filter[2,1,:,:]
    i = filter[2,2,:,:]

    fs1 = expand_dims(a+c+g+i, 0)/4
    fs2 = expand_dims(b+d+f+h,0)/4
    fs3= expand_dims(e, 0)

    sym = stack([concat([fs1, fs2, fs1],  axis=0), 
                         concat([fs2, fs3, fs2], axis=0),
                         concat([fs1, fs2, fs1], axis=0)])
        
    anti = filter - sym

    return sym, anti


#### Experiment #1 : Anistym only in block5

In [6]:
512*512

262144

In [43]:
gc.collect()
model = VGG16(weights='imagenet',
				  include_top=True,
				  input_shape=(224, 224, 3))
conv_layers = []
for l in model.layers:
	if 'conv2d' in str(type(l)).lower():
		if l.kernel_size == (3,3):
			conv_layers.append(l)

for layer in conv_layers[-3:]:
	#layer = get_conv_layer(model, -3)
	filters, biases = layer.get_weights()
	sym, antisym = getSymAntiSymTF(filters)
	a_energy = reduce_euclidean_norm(antisym, axis=[0,1])**2
	s_energy = reduce_euclidean_norm(sym, axis=[0,1])**2
	total_energy  = reduce_euclidean_norm(filters, axis=[0,1])**2
	beta  = a_energy/total_energy
	print("BETA", beta[0,0])

	mixed_indicies = tf.where(tf.math.logical_and(beta > 0.3 , beta < 0.7 ))
	#r = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  
	#norm = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  

	#filters[:,:,mixed_indicies.numpy()[:,0],mixed_indicies.numpy()[:, 1]] = np.zeros((3,3,1))	
	layer.set_weights([filters, biases])

	'''layer = get_conv_layer(model, -2)
	filters, biases = layer.get_weights()
	sym, antisym = getSymAntiSymTF(filters)
	r = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  
	norm = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  
	f = tf.math.multiply((filters / norm) , r)
	layer.set_weights([f, biases])

	layer = get_conv_layer(model, -1)
	filters, biases = layer.get_weights()
	sym, antisym = getSymAntiSymTF(filters)
	r = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  
	norm = tf.sqrt(tf.reduce_sum(tf.square(filters), axis=[0,1]))  
	f = tf.math.multiply((filters / norm) , r)
	layer.set_weights([f, biases])'''

model.compile(loss='categorical_crossentropy',
			  metrics=['accuracy'])


BETA tf.Tensor(0.92020845, shape=(), dtype=float32)
BETA tf.Tensor(0.31845495, shape=(), dtype=float32)
BETA tf.Tensor(0.8186765, shape=(), dtype=float32)


In [44]:
a = filters[0,0,:,:]
b = filters[0,1,:,:]
c = filters[0,2,:,:]
d = filters[1,0,:,:]
e = filters[1,1,:,:]
f = filters[1,2,:,:]
g = filters[2,0,:,:]
h = filters[2,1,:,:]
i = filters[2,2,:,:]

fs1 = (a+c+g+i)/4
fs2 = (b+d+f+h)/4
fs3= e


total_e = tf.math.reduce_euclidean_norm(filters, axis=[0,1])**2
sym_e= 4*fs1**2. + 4*fs2**2. + fs3**2. 
anti_e = total_e - sym_e
beta = anti_e / total_e
dist = (beta - 0.5)**2
reg = 1-4*dist

In [39]:
beta 

<tf.Tensor: shape=(512, 512), dtype=float32, numpy=
array([[0.8186765 , 0.22181098, 0.01863944, ..., 0.0069037 , 0.8516732 ,
        0.99361306],
       [0.6345305 , 0.6140087 , 0.0287166 , ..., 0.8402725 , 0.8358974 ,
        0.57847977],
       [0.20562099, 0.938982  , 0.36317477, ..., 0.35349146, 0.4544615 ,
        0.49786338],
       ...,
       [0.46263227, 0.6881922 , 0.9081367 , ..., 0.17629305, 0.7439231 ,
        0.04354893],
       [0.4872579 , 0.9763647 , 0.46864447, ..., 0.8667311 , 0.8501377 ,
        0.34095222],
       [0.3247266 , 0.2211252 , 0.25607044, ..., 0.14881541, 0.24192235,
        0.85003597]], dtype=float32)>

In [110]:
# Imagenet Validation Set location
img = "C:/ILSVRC2012_img_val/ILSVRC2012_val_00000028.JPEG"

In [111]:
train_list = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in train_labels], num_classes = 1000)


trainDS = Dataset.from_tensor_slices((train_list, train_labels))
trainDS = (trainDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(20)
	.prefetch(AUTOTUNE)
)


val_list = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in val_list], num_classes = 1000)


valDS = Dataset.from_tensor_slices((val_list, val_labels))
valDS = (valDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(20)
	.prefetch(AUTOTUNE)
)

In [107]:
np.argmax(train_labels[0, :])

0

In [113]:
model.evaluate(valDS.take(40))

40/40 [==============================] - 8s 169ms/step - loss: 2.8276 - accuracy: 0.4650


[2.8275744915008545, 0.4650000035762787]